## 機械学習モデルで意思決定をサポートする
Responsible AI Toolbox を用いて家の価格を向上させるための分析を行います。元データは [Housing Prices Dataset](https://www.kaggle.com/alphaepsilon/housing-prices-dataset) を用いています。列情報は [Data Description](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data) で確認できます。

### アジェンダ
0. 事前設定
1. ライブラリのインポート
2. データ準備
3. モデル学習
4. Responsible AI Dashboard 構築

<br>

### 事前設定
- Jupyter Kernel :  `raitoolbox` を選択する。
    - [setup.ipynb](../setup.ipynb) の手順に Python 環境のセットアップを完了しておくこと。

<br>

### 1. ライブラリ
データの前処理やモデル学習に必要な Python ライブラリをインポートします。

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor

<br>

### 2. データ準備
住宅の価格に関するデータを Pandas DataFrame としてインポートします。

In [ ]:
all_data = pd.read_csv('../data/apartments-train.csv')
all_data = all_data.drop(['Sold_HigherThan_Median','SalePrice'], axis=1)

In [ ]:
all_data.head()

In [ ]:
all_data.describe()

<br>

### 3. モデル構築

#### scikit-learn パイプラインの作成

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

def split_label(dataset, target_feature):
    X = dataset.drop([target_feature], axis=1)
    y = dataset[[target_feature]]
    return X, y

def clean_data(X, y, target_feature):
    features = X.columns.values.tolist()
    classes = y[target_feature].unique().tolist()
    pipe_cfg = {
        'num_cols': X.dtypes[X.dtypes == 'int64'].index.values.tolist(),
        'cat_cols': X.dtypes[X.dtypes == 'object'].index.values.tolist(),
    }
    num_pipe = Pipeline([
        ('num_imputer', SimpleImputer(strategy='median'))#,
        #('num_scaler', StandardScaler())
    ])
    cat_pipe = Pipeline([
        ('cat_imputer', SimpleImputer(strategy='constant', fill_value='?')),
        ('cat_encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
    ])
    feat_pipe = ColumnTransformer([
        ('num_pipe', num_pipe, pipe_cfg['num_cols']),
        ('cat_pipe', cat_pipe, pipe_cfg['cat_cols'])
    ])
    X = feat_pipe.fit_transform(X)
    print(pipe_cfg['cat_cols'])
    return X, feat_pipe, features, classes

In [ ]:
target_feature = 'SalePriceK'
categorical_features = []

X, y = split_label(all_data, target_feature)

# 学習データとテストデータに分割
X_train_original, X_test_original, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1234)

X_train, feat_pipe, features, classes = clean_data(X_train_original, y_train, target_feature)
y_train = y_train[target_feature].to_numpy()

X_test = feat_pipe.transform(X_test_original)
y_test = y_test[target_feature].to_numpy()

train_data = X_train_original.copy()
train_data[target_feature] = y_train

test_data = X_test_original.copy()
test_data[target_feature] = y_test

LightGBM を用いたモデル学習を実行します。

In [ ]:
clf = LGBMRegressor(n_estimators=5)
model = clf.fit(X_train, y_train)

<br>

### 4. Responsible AI Dashboard 構築

In [ ]:
from raiwidgets import ResponsibleAIDashboard
from responsibleai import RAIInsights

In [ ]:
from sklearn.pipeline import Pipeline

dashboard_pipeline = Pipeline(steps=[('preprocess', feat_pipe), ('model', model)])
rai_insights = RAIInsights(dashboard_pipeline, train_data, test_data, target_feature, 'regression',
                             categorical_features=categorical_features)

In [ ]:
# モデル説明性 (InterpretML)
rai_insights.explainer.add()
# モデル誤差解析 (Error Analysis)
rai_insights.error_analysis.add()
# 反実仮想サンプルの生成 (DiCE)
rai_insights.counterfactual.add(total_CFs=10, desired_range=[50, 1000])
# 因果推論 (EconML)
rai_insights.causal.add(treatment_features=['OverallCond', 'OverallQual', 'Fireplaces', 'GarageCars', 'ScreenPorch'])

インプットした情報をベースに計算をします。

In [ ]:
rai_insights.compute()

Responsible AI Toolbox ダッシュボードを生成します。表示される URI にブラウザからアクセスします。

In [ ]:
ResponsibleAIDashboard(rai_insights, locale="ja")